In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
data_path='../data/KGHM_clear.csv'

In [ ]:
df=pd.read_pickle(data_path)

In [ ]:
df.head()

In [ ]:
df_part=df.loc[:100,:]

In [ ]:
ax1 = df.plot(x='Date',y='Open',c='DarkBlue',legend=False)

This method is very slow

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(df['Date'], df['Open'])
# plt.show() # Depending on whether you use IPython or interactive mode, etc.

In [ ]:
ax1 = df.plot(x='Date',y='Open',c='DarkBlue',legend=False)

In [ ]:
df.LowHighDiff[df["LowHighDiff"]!=0].plot.hist(bins=20)
plt.show()

In [ ]:
df.OpenCloseDiff[df["OpenCloseDiff"]!=0].plot.hist(bins=20)
plt.show()

In [ ]:
df.LowHighDiff[df["LowHighDiff"]!=0].plot.hist(bins=20)
plt.show()

In [ ]:
fig, ax = plt.subplots()
df.TimeDelta.hist(ax=ax, bins=100, bottom=0.1)
ax.title()
ax.set_yscale('log')
fig,ax2=plt.subplots()
df.OpenCloseDiff.hist(ax=ax2, bins=100, bottom=0.1)
ax2.set_yscale('log')

In [ ]:
df.OpenCloseDiff.sum()

In [ ]:
df.LowHighDiff.sum()

In [ ]:
fig, ax = plt.subplots()
df.CoursChange.hist(ax=ax,bins=100, bottom=0.1)

# series.hist( bins=100, bottom=0.1)
ax.set_yscale('log')
ax.set_title('CourseChange')
plt.show()

In [ ]:
fig, ax = plt.subplots()
df.TimeDelta.hist(ax=ax,bins=100, bottom=0.1)

# series.hist( bins=100, bottom=0.1)
ax.set_yscale('log')
ax.set_title('TimeDelta')
plt.show()

In [ ]:
df[["CoursChange","TimeDelta"]].corr()

Jak widać, korelacja jest bardzo mała. Modele liniowe mogą działać słabo

In [ ]:
df2=df
df2=df2[["CoursChange","TimeDelta"]]

# This formats the plots such that they appear on separate rows
fig,axes= plt.subplots(2,1)
fig.tight_layout()

# Plot the PDF
df2.CoursChange.plot(ax=axes[0],kind='hist', normed=True, bins=30,title="CoursChange")
# plt.show()
# plt.title("CoursChange")
# plt.subplot(212)
df2.TimeDelta.plot(ax=axes[1],kind='hist', normed=True, bins=20,title="TimeDelta")

# Plot the CDF
# df2.CoursChange.plot(ax=axes[1], kind='hist', normed=True, cumulative=True, bins=30)
plt.show()


Zdecydowana większość zmian czasu i kursu jest bliska zeru

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

x = df.TimeDelta
data = [go.Histogram(x=x,
                     cumulative=dict(enabled=True),histnorm='probability',autobinx = False,xbins={'end':2500, 'size':1, 'start':0}
    )]

In [ ]:
layout = go.Layout(
#     title='Sampled Results',
    yaxis=dict(
        title='Probability'
    ),
    xaxis=dict(
        title='CourseChange'
    ),
    bargap=0.2,
    bargroupgap=0.1
)

fig=go.Figure(data=data,layout=layout)
py.iplot(fig, filename='TimeDeltaCDF')

In [ ]:
x = df[2000000:df.CoursChange.shape[0]].CoursChange
data = [go.Histogram(x=x,
                     cumulative=dict(enabled=True),histnorm='probability',autobinx = False,xbins={'end':6.015, 'size':0.005, 'start':-4}
    )]

In [ ]:
layout = go.Layout(
#     title='Sampled Results',
    yaxis=dict(
        title='Probability'
    ),
    xaxis=dict(
        title='CourseChange'
    ),
    bargap=0.2,
    bargroupgap=0.1
)

fig=go.Figure(data=data,layout=layout)
py.iplot(fig, filename='CoursChangeCDF')

In [ ]:
df[df["TimeDelta"]>800000].index

In [ ]:
df.iloc[3371819]

In [ ]:
df.iloc[3371820]

Anomalia pomiędzy 2014-01-31 09:00:01 a  2014-01-15 17:04:08 . Brak danych przez 15 dni

In [ ]:
df[df["OpenCloseDiff"]>4]

In [ ]:
df.loc[4901630:4901750][["Open"]].plot()

Dziwna zmiana ceny, notowania z poprzedniego dnia się urywają, duże różnice pomiędzy low and high

# Proba prognozowania

In [ ]:
df=pd.read_pickle(data_path)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, model_selection, svm

In [ ]:
df.head()

In [ ]:
df=df[['Close','TimeDelta']]

In [ ]:
df.head(15)

In [ ]:
df.shape

In [ ]:
data_size=5000000

In [ ]:
forecast_out=10

In [ ]:
df_test=df[data_size-5:data_size+forecast_out]

In [ ]:
df=df[:data_size]

In [ ]:
df.tail()

In [ ]:
df_test

In [ ]:
df['PredictionPrice'] = df[['Close']].shift(-forecast_out)
df['PredictionTime'] = df[['TimeDelta']].shift(-forecast_out)

In [ ]:
df.tail(15)

In [ ]:
# X = np.array(df.drop(['PredictionPrice',"PredictionTime"], 1))
X = np.array(df.drop(['PredictionPrice',"PredictionTime","TimeDelta"], 1))


In [ ]:
X

In [ ]:
X_forecast = X[-forecast_out:] # set X_forecast equal to last 30
X = X[:-forecast_out] # remove last 30 from X
X.shape

In [ ]:
X.shape

In [ ]:
X_forecast.shape

In [ ]:
# y = np.array(df[['PredictionPrice',"PredictionTime"]])
y =  np.array(df.drop(['Close',"PredictionTime","TimeDelta"], 1))
y = y[:-forecast_out]

In [ ]:
y.shape

In [ ]:
X

In [ ]:
# Create the Scaler object
scaler = preprocessing.StandardScaler()

In [ ]:
normalized_X = preprocessing.scale(X)

In [ ]:
normalized_y = preprocessing.scale(y)

In [ ]:
print(normalized_y.shape,normalized_X.shape)

In [ ]:
normalized_X

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(normalized_X, normalized_y, test_size = 0.2, shuffle=False)

In [ ]:
X_train,y_train

In [ ]:
# Training
clf = LinearRegression()
clf.fit(X_train,y_train)
# Testing
confidence = clf.score(X_test, y_test)
print("confidence: ", confidence)

In [ ]:
forecast_prediction = clf.predict(X_forecast)
print(forecast_prediction)

In [ ]:
X_forecast

In [ ]:
df_test